# TensorFlow 2.0 alpha - tf.function
### Tool for making Graphs out of programs

In [1]:
from __future__ import absolute_import, print_function, unicode_literals, division

import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### a Function is like an operation

In [2]:
@tf.function
def add(a,b) :
    return a + b

add(tf.ones([2,2]), tf.ones([2,2]))

<tf.Tensor: id=16, shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

#### Functions have Gradients

In [3]:
@tf.function
def add(a,b):
    return a+b

v = tf.Variable(1.0)
with tf.GradientTape() as tape:
    result = add(v, 1.0)
tape.gradient(result, v)

<tf.Tensor: id=44, shape=(), dtype=float32, numpy=1.0>

#### Can use Functions, inside Functions

In [4]:
@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3,2]), tf.ones([2,2]), tf.ones([2]))

<tf.Tensor: id=74, shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

## Polymorphism
#### tf.function can be as flexible and generic as python functions
#### Call a function with arguments of different type - Functions are Polymorphic

In [5]:
@tf.function
def add(a):
    return a+a

print('add 1', add(1))
print('add 1.1', add(1.1))
print('add string tensor', add(tf.constant('a')))

c = add.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))
c(a = tf.constant('a')) 

add 1 tf.Tensor(2, shape=(), dtype=int32)
add 1.1 tf.Tensor(2.2, shape=(), dtype=float32)
add string tensor tf.Tensor(b'aa', shape=(), dtype=string)


<tf.Tensor: id=104, shape=(), dtype=string, numpy=b'aa'>

#### Functions can be Faster than eager code - for Graphs with small operations

In [7]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
    return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])

conv_layer(image); conv_fn(image)

print('Eager conv:', timeit.timeit(lambda: conv_layer(image), number=10))
print('Function conv:', timeit.timeit(lambda: conv_fn(image), number=10))
print()
print("Note how there's not much difference in performance for convolutions")
print()

lstm_cell = tf.keras.layers.LSTMCell(10)

@tf.function
def lstm_fn(input, state):
    return lstm_cell(input, state)

input = tf.zeros([10,10])
state = [tf.zeros([10,10])] * 2

lstm_cell(input, state); lstm_fn(input, state)

print('Eager lstm:', timeit.timeit(lambda: lstm_cell(input, state), number=10))
print('Function lstm:', timeit.timeit(lambda: lstm_fn(input, state), number=10))

Eager conv: 0.8870195341296494
Function conv: 0.7944547641091049

Note how there's not much difference in performance for convolutions

Eager lstm: 0.04588328907266259
Function lstm: 0.003938127309083939


### State in tf.function
#### There is no need to add manual control dependencies - tf.function can handle the intended execution order

In [8]:
# automatic control dependencies

a = tf.Variable(1.0)
b = tf.Variable(2.0)

@tf.function
def f(x, y):
    a.assign(y*b)
    b.assign_add(x*a)
    return a+b

f(1.0, 2.0) 

<tf.Tensor: id=3083, shape=(), dtype=float32, numpy=10.0>

### Variables
#### Downside - with ambiguous code, different behavior can occur with multiple eager calls leading to over-evaluation of output tensor

In [9]:
# example

@tf.function
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return v

f(1.)                 # Broken - Leads to ERROR

ValueError: tf.function-decorated function tried to create variables on non-first call.

#### Repeatedly evaluating the Tensor obtained from f(1.) - in a graph context, causes you to get increasing numbers
#### tf.function does not allow this - NON-ambiguous code works

In [10]:
v = tf.Variable(1.0)

@tf.function
def f(x):
    return v.assign_add(x)

print(f(1.0)),
print(f(2.0))

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


#### Variables can be created Inside Functions - ONLY if, the variable is executed once, upon initial Function execution

In [11]:
class C: pass
obj = C(); obj.v = None

@tf.function
def g(x):
    if obj.v is None:
        obj.v = tf.Variable(1.0)
    return obj.v.assign_add(x)

print(g(1.0)),
print(g(2.0))

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


#### Variable Initializers - can depend on Function arguments, or on other variables
#### Figure out the right Initialization Order - using same method to generate Control Dependencies

In [12]:
state = []

@tf.function
def fn(x):
    if not state:
        state.append(tf.Variable(2.0 * x))
        state.append(tf.Variable(state[0] * 3.0))
    return state[0] * x * state[1]

fn(tf.constant(1.0))
fn(tf.constant(3.0))

W0411 19:46:03.033680 140736985473984 tf_logging.py:161] Entity <method-wrapper '__call__' of weakref object at 0xb32203638> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable AUTOGRAPH_VERBOSITY >= 1. Please report this to the AutoGraph team. Cause: Object conversion is not yet supported. If you are trying to convert code that uses an existing object, try including the creation of that object in the conversion. For example, instead of converting the method of a class, try converting the entire class instead. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/README.md#using-the-functional-api for more information.
W0411 19:46:03.041373 140736985473984 tf_logging.py:161] Entity <method-wrapper '__call__' of weakref object at 0xb322ae0e8> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable A

<tf.Tensor: id=3309, shape=(), dtype=float32, numpy=36.0>

## Control Flow and Autograph
#### tf.cond and tf.while_loop will work with tf.function
#### plus, the AUTOGRAPH library - will rewrite conditionals and loops, which depend on Tensors to run the graph

In [13]:
# Simple loop

@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

f(tf.random.uniform([10]))

[0.418310046 0.128251672 0.0406667 ... 0.480195522 0.935450315 0.897618651]
[0.395505756 0.127553076 0.0406442918 ... 0.446400195 0.733124912 0.715136409]
[0.376096964 0.126865789 0.0406219214 ... 0.418935478 0.624973476 0.613887668]
[0.35931313 0.1261895 0.0405995883 ... 0.396033227 0.554581344 0.546857953]
[0.344608843 0.125523925 0.0405772962 ... 0.376549721 0.503945947 0.498161614]
[0.331585735 0.12486878 0.0405550338 ... 0.359707326 0.465214729 0.460670143]
[0.319944859 0.124223806 0.0405328088 ... 0.344956189 0.434324801 0.430630237]
[0.309457064 0.123588733 0.0405106209 ... 0.331894845 0.408929229 0.405847877]
[0.299943089 0.122963309 0.0404884703 ... 0.320222348 0.387563139 0.384941489]
[0.291260511 0.122347288 0.0404663607 ... 0.30970794 0.36925751 0.366991162]
[0.283294529 0.121740438 0.040444281 ... 0.300171345 0.353342086 0.351357073]
[0.275951475 0.121142536 0.0404222421 ... 0.291469395 0.339336127 0.337578475]
[0.269153923 0.120553367 0.0404002368 ... 0.283486664 0.326884

<tf.Tensor: id=3354, shape=(10,), dtype=float32, numpy=
array([0.12504108, 0.09172215, 0.03884573, 0.01900934, 0.08868066,
       0.12045684, 0.12810788, 0.12638417, 0.12959203, 0.12949447],
      dtype=float32)>

#### Inspect the code Autograph generates

In [14]:
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

print(tf.autograph.to_code(f))

from __future__ import print_function

def tf__f(x):
  try:
    with ag__.function_scope('f'):
      do_return = False
      retval_ = None

      def loop_test(x_1):
        with ag__.function_scope('loop_test'):
          return ag__.gt(ag__.converted_call('reduce_sum', tf, ag__.ConversionOptions(recursive=True, verbose=0, strip_decorators=(ag__.convert, ag__.do_not_convert, ag__.converted_call), force_conversion=False, optional_features=ag__.Feature.ALL, internal_convert_user_code=True), (x_1,), {}), 1)

      def loop_body(x_1):
        with ag__.function_scope('loop_body'):
          with ag__.utils.control_dependency_on_returns(ag__.converted_call('print', tf, ag__.ConversionOptions(recursive=True, verbose=0, strip_decorators=(ag__.convert, ag__.do_not_convert, ag__.converted_call), force_conversion=False, optional_features=ag__.Feature.ALL, internal_convert_user_code=True), (x_1,), {})):
            x, tf_1 = ag__.utils.alias_tensors(x_1, tf)
            x = ag__.converted_call(

### Controlling Autograph
#### It only affects basic Control Flow constructs in python (if, while, for, etc.) - Only changing them, if the predicates are Tensors

In [15]:
@tf.function
def f(x):
    for i in range(10):          # static python loop - unrolled, no conversion
        do_stuff()
    for i in tf.range(10):       # depends on a Tensor - will be converted

    
# Will Get ERROR

SyntaxError: unexpected EOF while parsing (<ipython-input-15-bfcdc0e4d7b8>, line 5)

#### In addition, Guarantee prints and asserts happen dynamically - using tf.print, and tf.assert

In [16]:
@tf.function
def f(x):
    for i in tf.range(10):
        tf.print(i)
        tf.Assert(i < 10, ['a'])
        x += x
    return x

f(10)

0
1
2
3
4
5
6
7
8
9


<tf.Tensor: id=3417, shape=(), dtype=int32, numpy=10240>

#### Autograph cannot compile python into TF Graphs - data structures, to be used dynamically, Must be TensorFlow data structures
### tf.TensorArray 
#### This method would be best to accumulate data in a loop

In [17]:
@tf.function
def f(x):
    ta = tf.TensorArray(tf.float32, size=10)
    for i in tf.range(10):
        x += x
        ta = ta.write(i, x)
    return ta.stack()

f(10.0)

<tf.Tensor: id=3486, shape=(10,), dtype=float32, numpy=
array([   20.,    40.,    80.,   160.,   320.,   640.,  1280.,  2560.,
        5120., 10240.], dtype=float32)>

# Conclusion
### Functions can give the runtime more information about what was the intended behavior of the code. tf.function is capable of adding the minimal set of necessary Control Dependencies for the code to run correctly. Often for certain dynamic operations, Autograph (fully integrated library with tf.function) requires TensorFlow data structures (tf._)_ rather than simple python structures